In [ ]:
from os import environ
from flow_handler import flow_handler
import requests as r
import pandas as pd
import logging

In [ ]:
INSTANCE = "seap"
# SURVEY ID CAN BE FOUND ON https://webform.akvotest.org/
SURVEY_ID = 290080914
FORM_ID = 300160915

In [ ]:
baseURI = f'https://api-auth0.akvo.org/flow/orgs/{INSTANCE}'
envs = ["AUTH0_CLIENT_ID","FLOW_USERNAME","FLOW_PASSWORD"]
for env in envs:
    if env not in environ:
        logging.error(f"{env} NOT FOUND")

In [ ]:
def get_access_token():
    tokenURI = 'https://akvofoundation.eu.auth0.com/oauth/token'
    auth = {
        "client_id": environ['AUTH0_CLIENT_ID'],
        "username": environ['FLOW_USERNAME'],
        "password": environ['FLOW_PASSWORD'],
        "grant_type": "password",
        "scope": "openid email",
    }

    try:
        account = r.post(tokenURI, data=auth).json();
    except:
        logging.error('FAILED TO REQUEST TOKEN')
        return False
    return account["id_token"]

In [ ]:
def get_response(token, url):
    headers = {
        "Content-Type":"application/json",
        "Accept": "application/vnd.akvo.flow.v2+json",
        "Authorization": "Bearer {}".format(token)
    }
    response = r.get(url, headers=headers).json()
    return response

In [ ]:
def get_folders(token, parent_id = 0):
    return get_response(token, f"{baseURI}/folders?parent_id={parent_id}")

In [ ]:
def get_surveys(token, folder_id = 0):
    return get_response(token, f"{baseURI}/surveys?folder_id={folder_id}").get("surveys")

In [ ]:
def get_forms(token, survey_id):
    return get_response(token, f"{baseURI}/surveys/{survey_id}").get("forms")

In [ ]:
def get_form_instances(token, survey_id, form_id, next_page_url=False, result=[]):
    url = f"{baseURI}/form_instances?survey_id={survey_id}&form_id={form_id}"
    if next_page_url:
        url = next_page_url
    res = get_response(token, url)
    result += res.get("formInstances")
    if res.get("nextPageUrl"):
        get_form_instances(token, survey_id, form_id, res.get("nextPageUrl"), result)
    return result

In [ ]:
token = get_access_token()

In [ ]:
forms = get_forms(token, SURVEY_ID)
form_instances = get_form_instances(token, SURVEY_ID, FORM_ID)

In [ ]:
form = list(filter(lambda x: int(x["id"]) == FORM_ID, forms))[0]

In [ ]:
question_group_list = {}
question_list = {}
for qg in form.get("questionGroups"):
    question_group_list.update({qg["id"]:{"name": qg["name"],"repeatable": qg["repeatable"]}})
    for q in qg.get("questions"):
        question_list.update({q["id"]: {"name": q["name"],"type": q["type"]}})
collections = []
for form_instance in form_instances:
    print("___________________________________________________\n")
    metadata = {}
    for meta in form_instance:
        if meta != "responses":
            metadata.update({meta: form_instance[meta]})
            print(f"{meta}: {form_instance[meta]}")
    for qg in question_group_list:
        group_name = question_group_list[qg]["name"]
        print(f"\nGROUP: {group_name}\n")
        data = metadata
        data.update({"group": group_name})
        question_group_responses = form_instance.get("responses").get(qg)
        if question_group_responses:
            for repeat, responses in enumerate(question_group_responses):
                if question_group_list[qg]["repeatable"]:
                    data.update({"repeat_index": repeat})
                    print(f"  REPEAT: {repeat}\n")
                answers = {}
                for qid in responses:
                    question = question_list[qid]
                    answer = responses[qid]
                    answer = flow_handler(answer, question.get("type"))
                    question_name = question.get("name")
                    answers.update({question_name: answer})
                    print(f"  Q:{question_name}\n  A:{answer}\n")
                data.update(answers)
                collections.append(data)